pip install -r requirements.txt (가상환경에 이미 설치됨)

주피터노트북 커널 연결(python -m ipykernel install --user --name myenv --display-name corpus)

한글 폰트(주피터랩 자체를 다시 켜야지 폰트 적용되는 것 같음.)

apt-get update

apt-get install -y fonts-nanum

# bareun download(bareunpy-1.6.6)

In [2]:
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

 2022신문코퍼스_NLTK.ipynb		 bareun-linux.deb
'250325_data processing_bareun0.ipynb'	 corpus_statistics_df0.ipynb
'250325_data processing_bareun1.ipynb'	 myenv
'250325_data processing_bareun2.ipynb'	 nohup.out
 NIKL_NEWSPAPER_16.csv			 requirements.txt
 NIKL_NEWSPAPER_16.parquet		 tagged_NIKL_NEWSPAPER_PARQUET
 NIKL_NEWSPAPER_PARQUET


In [3]:
!uname -a
!dpkg -i bareun-linux.deb

Linux 372f2dc7b32b 6.8.0-45-generic #45~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Sep 11 15:25:05 UTC 2 x86_64 x86_64 x86_64 GNU/Linux
(Reading database ... 20752 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
Unpacking bareun (3.0.rc1) ...
Setting up bareun (3.0.rc1) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Configuration file /etc/systemd/system/bareun.service is marked executable. Please remove executable permission bits. Proceeding anyway.
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


In [4]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  37.3M      0  0:00:10  0:00:10 --:--:-- 41.0M


In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

In [5]:
!ps -ef | grep bareun

root        3181    2266 99 19:44 pts/5    00:00:18 /opt/bareun/bin/bareun
root        3586    2210  0 19:44 pts/6    00:00:00 /usr/bin/sh -c ps -ef | grep bareun
root        3588    3586  0 19:44 pts/6    00:00:00 grep bareun


In [7]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg "koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA"

Use config file: /opt/bareun/config/bareun.json

------
200 OK
sucessfully registered koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA


# library

In [8]:
import bareunpy as brn
import google.protobuf.text_format as tf
import sys
import tqdm
import pandas as pd
import pyarrow
import fastparquet

# bareun test

In [9]:
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"
API_KEY = "koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA" # <- 본인의 API KEY로 교체
t = brn.Tagger(API_KEY, "localhost", 5656) # tagger는 변수 t에 할당
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."]) # t.tags()에 리스트로 한 문장을 입력
m = res.msg() # 분석 결과를 객체로 저장하여 변수 m에 할당

tf.PrintMessage(m, out=sys.stdout, as_utf8=True) # 분석 결과 객체 m을 가독성 좋게 표시.

sentences {
  text {
    content: "안녕하세요. 정말 좋은 날씨네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.99983704
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9785348
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.9992842
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "정말"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "정말"
        begin_offset: 7
        length: 2
      }
      tag: MAG
      probabili

# 데이터 확인

In [10]:
def tag_sentences_with_bareun(df, api_key, host='localhost', port=5656):
    # Bareun Tagger 초기화
    t = brn.Tagger(api_key, host, port)

    # 총 문장 수 출력
    print(f"Total number of sentences to tag: {len(df)}")
    
    # 각 문장 태깅 함수
    def tag_sentence(sentence):
      try:
          res = t.tags([sentence])
          return list(res.pos())
      except Exception as e:
          print(f"Error tagging sentence: {sentence}")
          print(f"Error: {e}")
          return []

    tagged_sentences = []
    for sentence in tqdm.tqdm(df['sentence'], desc="Tagging Sentences", mininterval=10):
        tagged_sentences.append(tag_sentence(sentence))
    
    # 새로운 'tagged_sentence' 열 추가
    df['tagged_sentence'] = tagged_sentences
    
    return df

In [19]:
df = pd.read_parquet('/workspace/NIKL_NEWSPAPER_PARQUET/NIKL_NEWSPAPER_35.parquet')

In [ ]:
tagged_df = tag_sentences_with_bareun(df, API_KEY)

Total number of sentences to tag: 1000000


Tagging Sentences:  94%|█████████▍| 939731/1000000 [4:15:26<15:58, 62.91it/s]  

In [ ]:
# parquet 파일로 저장
tagged_df.to_parquet('/workspace/tagged_NIKL_NEWSPAPER_PARQUET/tagged_NIKL_NEWSPAPER_35.parquet', index=False)

# convert csv to parquet

In [ ]:
def convert_csv_to_parquet_pandas(csv_path, parquet_path):
    """
    Pandas를 사용하여 CSV를 Parquet로 변환
    """
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)
    
    # Parquet 파일로 저장
    df.to_parquet(parquet_path, index=False)

In [ ]:
def convert_multiple_csv_to_parquet(start_num, end_num):
    for i in range(start_num, end_num + 1):
        input_csv = f'/workspace/NIKL_NEWSPAPER_CSV/NIKL_NEWSPAPER_{i}.csv'
        output_parquet = f'/workspace/NIKL_NEWSPAPER_PARQUET/NIKL_NEWSPAPER_{i}.parquet'
        
        convert_csv_to_parquet_pandas(input_csv, output_parquet)
        print(f"Converted {input_csv} to {output_parquet}")

In [ ]:
convert_csv_to_parquet_pandas('/workspace/NIKL_NEWSPAPER_16.csv', '/workspace/NIKL_NEWSPAPER_16.parquet')